In [ ]:
import random

def generate(length, l=10, negative_random_num=0, negative_reps_num=0, positive_end_num=0, positive_random_num=0):
    chars = ['A', 'C', 'G', 'T']
    result = ""
    words = []
    current_pos = 0
    
    which_to_repeat = []
    where_to_repeat = []
    which_where = {}  
    
    
    #budowanie łańcucha bez powtórzeń
    while(current_pos < length):
        new_char = random.choice(chars)
        new_word = result[-(l-1):] + new_char
        i = 0
        
        if(current_pos >= (l-1)):     
            while(new_word in words):
                new_char = chars[i]
                new_word = result[-(l-1):] + new_char
                i += 1
                if(i == 4): #każda litera powoduje powtórzenie -> cofamy się o l znaków
                    current_pos -= l
                    result = result[:-l]
                    words = words[:-l]                    
                    break
                
            if(i == 4):
                continue
            
            words.append(new_word)        
            
        current_pos += 1    
        result += new_char
        
    
    #losowanie indeksów z których słowa zostaną powtórzone
    while(negative_reps_num):
        which_idx = random.randint(0, length - l)
        if(which_idx not in which_to_repeat):
            which_to_repeat.append(which_idx)
            negative_reps_num -= 1
        
    #losowanie indeksów na których pojawi się powtórzenie
    for i in range(len(which_to_repeat)):
        where_idx = random.randint(0, length - l)
        while(where_idx in which_to_repeat or any([abs(where_idx - w) < l for w in where_to_repeat]) or any([abs(where_idx - w) < l for w in which_to_repeat])):
            where_idx = random.randint(0, length - l)

        where_to_repeat.append(where_idx)
        which_where[where_idx] = random.choice(which_to_repeat)  
        
    #celowe powtórzenie słów
    for target, source in which_where.items():
        words[target] = words[source]
        
        #poprawienie słów przed i po, żeby zawierały powtórzony podciąg
        for i in range(1, l):
            if(target - i >= 0):
                words[target - i] = words[target - i][:i] + words[source][:-i]
            if(target + i <= length - l):
                words[target + i] = words[source][i:] + words[target + i][-i:]
     
    actual_number_of_reps = len(words) - len(set(words))
    
        
    #błędy losowe negatywne
    to_delete = []
    while(negative_random_num):
        idx = random.randint(0, len(words) - 1)
        #żeby nie wywalić powtórzonych
        while(idx in which_to_repeat or idx in where_to_repeat):
            idx = random.randint(0, len(words) - 1)
        
        to_delete.append(idx)
        negative_random_num -= 1
        
    words = [w for i, w in enumerate(words) if i not in to_delete]
    
        
    #błędy pozytywne z przekłamań na końcach
    while(positive_end_num):
        idx = random.randint(0, len(words) - 1)
        word = words[idx]
        
        new_end = word[-1]
        while(new_end == word[-1]):
            new_end = random.choice(chars)
            
        new_word = word[:-1] + new_end
        
        if(new_word not in words):
            words.append(new_word)
            positive_end_num -= 1
            
            
    #błędy pozytwne z szumu
    while(positive_random_num):
        new_word = ""
        chars_to_gen = l
        while(chars_to_gen):
            new_word += random.choice(chars)
            chars_to_gen -= 1
            
        #sprawdzenie czy nasze losowe słowo nie jest jakimś właściwym
        if(new_word not in words):
            words.append(new_word)
            positive_random_num -= 1

            return words, actual_number_of_reps

In [ ]:
#Jest pewna szansa, że wpadnie w jakąś dziką pętlę przy powtórzeniach.
#Jeżeli trwa to minutę, to przerwij i spróbuj jeszcze raz albo zmniejsz liczbę powtórzeń
# MODYFIKUJ WARTOŚCI TUTAJ
length = 1000
l = 10
negative_random_num = 0
negative_reps_num = 5
positive_end_num = 0
positive_random_num = 0

res, negative_reps_num = generate(length, l, negative_random_num, negative_reps_num, positive_end_num, positive_random_num)
print(negative_reps_num)

# PRETTY PRINT
# spaces = " "
# for i in range(len(res)):
#     print(str(i).rjust(4, '0') + spaces + res[i])
#     spaces += " "

#posortowanie i wybranie unikatowych
res = sorted(set(res))

file = open("instances/custom/" + str(length) + "_" + str(l) + "-" + str(negative_random_num) + "-" + str(negative_reps_num) + "+" + str(positive_end_num) + "+" + str(positive_random_num),"w")  
for r in res:
    file.write(r + "\n")
file.close() 